<a href="https://colab.research.google.com/github/taxuyou/library-recommend-and-timeseries-predict/blob/master/%ED%99%88%ED%8E%98%EC%9D%B4%EC%A7%80_%EB%93%B1%EB%A1%9D%EC%9A%A9_%EB%B0%A9%EB%AC%B8%EC%9E%90_lstm_%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [0]:
# 랜덤에 의해 똑같은 결과를 재현하도록 시드 설정
# 하이퍼파라미터를 튜닝하기 위한 용도(흔들리면 무엇때문에 좋아졌는지 알기 어려움)
tf.set_random_seed(777)

In [0]:
# Standardization
def data_standardization(x):
    x_np = np.asarray(x)
    return (x_np - x_np.mean()) / x_np.std()

In [0]:
# 너무 작거나 너무 큰 값이 학습을 방해하는 것을 방지하고자 정규화한다
# x가 양수라는 가정하에 최소값과 최대값을 이용하여 0~1사이의 값으로 변환
# Min-Max scaling
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7) # 1e-7은 0으로 나누는 오류 예방차원
 
# 정규화된 값을 원래의 값으로 되돌린다
# 정규화하기 이전의 org_x값과 되돌리고 싶은 x를 입력하면 역정규화된 값을 리턴한다
def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()

In [0]:
# 하이퍼파라미터
input_data_column_cnt = 6  # 입력데이터의 컬럼 개수(Variable 개수)
output_data_column_cnt = 1 # 결과데이터의 컬럼 개수
 
seq_length = 7           # 1개 시퀀스의 길이(시계열데이터 입력 개수)
rnn_cell_hidden_dim = 20   # 각 셀의 (hidden)출력 크기
forget_bias = 1          # 망각편향(기본값 1.0)
num_stacked_layers = 2     # stacked LSTM layers 개수
keep_prob = 1.0            # dropout할 때 keep할 비율
 
epoch_num = 2000           # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01       # 학습률

In [0]:
# 데이터를 로딩한다.
from datetime import datetime
raw_dataframe = pd.read_csv('test4.csv', 
                    )

In [8]:
data_info = raw_dataframe.values[0:].astype(np.float) #  문자열을 부동소수점형으로 변환한다
print("data_info.shape: ", data_info.shape)
print("data_info[0]: ", data_info[0])

data_info.shape:  (939, 6)
data_info[0]:  [-2.600e+00  0.000e+00  5.000e-01  1.000e+00  0.000e+00  2.221e+03]


In [9]:
# 데이터들을 정규화한다
# ['days','holiday','workingday','closed', 'visitors']에서 'workingday'까지 취함
# 곧, 마지막 열 Volume를 제외한 모든 열
day = data_info[:,0:5]
norm_day = min_max_scaling(day) # 가격형태 데이터 정규화 처리
print("day.shape: ", day.shape)
print("day[0]: ", day[0])
print("norm_day[0]: ", norm_day[0])
print("="*100) # 화면상 구분용

day.shape:  (939, 5)
day[0]:  [-2.6  0.   0.5  1.   0. ]
norm_day[0]:  [0.089508   0.10491998 0.10788382 0.11084766 0.10491998]


In [10]:
# 방문자 데이터를 정규화한다
# ['days','holiday','workingday','visitors']에서 마지막 'visitors'만 취함
# [:,-1]이 아닌 [:,-1:]이므로 주의하자! 스칼라가아닌 벡터값 산출해야만 쉽게 병합 가능
visitor = data_info[:,-1:]
norm_visitor = min_max_scaling(visitor) # 거래량형태 데이터 정규화 처리
print("visitor.shape: ", visitor.shape)
print("vitisor[0]: ", visitor[0])
print("norm_visitor[0]: ", norm_visitor[0])
print("="*100) # 화면상 구분용
 
# 행은 그대로 두고 열을 우측에 붙여 합친다
x = np.concatenate((norm_day, norm_visitor), axis=1) # axis=1, 세로로 합친다
print("x.shape: ", x.shape)
print("x[0]: ", x[0])    # x의 첫 값
print("x[-1]: ", x[-1])  # x의 마지막 값
print("="*100) # 화면상 구분용
 
y = x[:, [5]] # 타켓은 방문자다
print("y[0]: ",y[0])     # y의 첫 값
print("y[-1]: ",y[-1])   # y의 마지막 값
 
 
dataX = [] # 입력으로 사용될 Sequence Data
dataY = [] # 출력(타켓)으로 사용


for i in range(0, len(y) - seq_length):
    _x = x[i : i+seq_length]
    _y = y[i + seq_length] # 다음 나타날 방문자수(정답)
    if i is 0:
        print(_x, "->", _y) # 첫번째 행만 출력해 봄
    dataX.append(_x) # dataX 리스트에 추가
    dataY.append(_y) # dataY 리스트에 추가

visitor.shape:  (939, 1)
vitisor[0]:  [2221.]
norm_visitor[0]:  [0.30558613]
x.shape:  (939, 6)
x[0]:  [0.089508   0.10491998 0.10788382 0.11084766 0.10491998 0.30558613]
x[-1]:  [0.25074096 0.21754594 0.11025489 0.14641375 0.10491998 0.53783709]
y[0]:  [0.30558613]
y[-1]:  [0.53783709]
[[0.089508   0.10491998 0.10788382 0.11084766 0.10491998 0.30558613]
 [0.09247184 0.10491998 0.11084766 0.11084766 0.10491998 0.33668134]
 [0.07824541 0.10491998 0.10966212 0.11084766 0.10491998 0.40767749]
 [0.08713693 0.10491998 0.10847659 0.11084766 0.10491998 0.36502477]
 [0.08891523 0.10491998 0.11203319 0.11084766 0.10491998 0.36144744]
 [0.09780676 0.10491998 0.10966212 0.11084766 0.10491998 0.36406164]
 [0.08298755 0.10491998 0.10847659 0.11084766 0.10491998 0.46037424]] -> [0.59039626]


In [0]:
# 학습용/테스트용 데이터 생성
# 전체 70%를 학습용 데이터로 사용
train_size = int(len(dataY) * 0.9777)

# 나머지(30%)를 테스트용 데이터로 사용
test_size = len(dataY) - train_size
 
# 데이터를 잘라 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])
 
# 데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

In [12]:
# 텐서플로우 플레이스홀더 생성
# 입력 X, 출력 Y를 생성한다
X = tf.placeholder(tf.float32, [None, seq_length, input_data_column_cnt])
print("X: ", X)
Y = tf.placeholder(tf.float32, [None, 1])
print("Y: ", Y)
 
# 검증용 측정지표를 산출하기 위한 targets, predictions를 생성한다
targets = tf.placeholder(tf.float32, [None, 1])
print("targets: ", targets)
 
predictions = tf.placeholder(tf.float32, [None, 1])
print("predictions: ", predictions)

X:  Tensor("Placeholder:0", shape=(?, 7, 6), dtype=float32)
Y:  Tensor("Placeholder_1:0", shape=(?, 1), dtype=float32)
targets:  Tensor("Placeholder_2:0", shape=(?, 1), dtype=float32)
predictions:  Tensor("Placeholder_3:0", shape=(?, 1), dtype=float32)


In [13]:
# 모델(LSTM 네트워크) 생성
def lstm_cell():
    # LSTM셀을 생성
    # num_units: 각 Cell 출력 크기
    # forget_bias:  to the biases of the forget gate 
    #              (default: 1)  in order to reduce the scale of forgetting in the beginning of the training.
    # state_is_tuple: True ==> accepted and returned states are 2-tuples of the c_state and m_state.
    # state_is_tuple: False ==> they are concatenated along the column axis.
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell
 
# num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [14]:
# RNN Cell(여기서는 LSTM셀임)들을 연결
hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)
 
# [:, -1]를 잘 살펴보자. LSTM RNN의 마지막 (hidden)출력만을 사용했다.
# 과거 여러 거래일의 주가를 이용해서 다음날의 주가 1개를 예측하기때문에 MANY-TO-ONE형태이다
hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], output_data_column_cnt, activation_fn=tf.identity)
 
 
# 손실함수로 평균제곱오차를 사용한다
loss = tf.reduce_sum(tf.square(hypothesis - Y))
# 최적화함수로 AdamOptimizer를 사용한다
optimizer = tf.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.RMSPropOptimizer(learning_rate) # LSTM과 궁합 별로임
 
train = optimizer.minimize(loss)
 
# RMSE(Root Mean Square Error)
# 제곱오차의 평균을 구하고 다시 제곱근을 구하면 평균 오차가 나온다
# rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions))) # 아래 코드와 같다
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))
 
 
train_error_summary = [] # 학습용 데이터의 오류를 중간 중간 기록한다
test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
test_predict = ''        # 테스트용데이터로 예측한 결과
 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 20), dtype=float32)
Instructions for updating:
Please use `layer.__call__` method instead.


In [15]:
# 학습한다
# start_time = datetime.datetime.now() # 시작시간을 기록한다
print('학습을 시작합니다...')
for epoch in range(epoch_num):
    _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
    if ((epoch+1) % 100 == 0) or (epoch == epoch_num-1): # 100번째마다 또는 마지막 epoch인 경우
        # 학습용데이터로 rmse오차를 구한다
        train_predict = sess.run(hypothesis, feed_dict={X: trainX})
        train_error = sess.run(rmse, feed_dict={targets: trainY, predictions: train_predict})
        train_error_summary.append(train_error)
 
        # 테스트용데이터로 rmse오차를 구한다
        test_predict = sess.run(hypothesis, feed_dict={X: testX})
        test_error = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
        test_error_summary.append(test_error)
        
        # 현재 오류를 출력한다
        print("epoch: {}, train_error(A): {}, test_error(B): {}, B-A: {}".format(epoch+1, train_error, test_error, test_error-train_error))
        
# end_time = datetime.datetime.now() # 종료시간을 기록한 다
# elapsed_time = end_time - start_time # 경과시간을 구한다
# print('elapsed_time:',elapsed_time)
# print('elapsed_time per epoch:',elapsed_time/epoch_num)

학습을 시작합니다...
epoch: 100, train_error(A): 0.14452990889549255, test_error(B): 0.135231614112854, B-A: -0.00929829478263855
epoch: 200, train_error(A): 0.11056042462587357, test_error(B): 0.10917548835277557, B-A: -0.001384936273097992
epoch: 300, train_error(A): 0.10188576579093933, test_error(B): 0.11121141910552979, B-A: 0.009325653314590454
epoch: 400, train_error(A): 0.10183723270893097, test_error(B): 0.09002826362848282, B-A: -0.01180896908044815
epoch: 500, train_error(A): 0.09040447324514389, test_error(B): 0.11323431879281998, B-A: 0.022829845547676086
epoch: 600, train_error(A): 0.08505728840827942, test_error(B): 0.11169756203889847, B-A: 0.02664027363061905
epoch: 700, train_error(A): 0.08241607248783112, test_error(B): 0.11699903011322021, B-A: 0.0345829576253891
epoch: 800, train_error(A): 0.07588328421115875, test_error(B): 0.11843493580818176, B-A: 0.04255165159702301
epoch: 900, train_error(A): 0.07224223762750626, test_error(B): 0.10908130556344986, B-A: 0.036839067935

In [16]:
# sequence length만큼의 가장 최근 데이터를 슬라이싱한다
recent_data = np.array([x[len(x)-seq_length : ]])
print("recent_data.shape:", recent_data.shape)
print("recent_data:", recent_data)
 
# 내일 방문자를 예측
test_predict1 = sess.run(hypothesis, feed_dict={X: recent_data})
 
print("test_predict1", test_predict1[0])
test_predict2 = reverse_min_max_scaling(visitor, test_predict1) # 금액데이터 역정규화한다
print("내일의 예측 ","visitors", test_predict2) # 예측한 visitor를 출력한다

recent_data.shape: (1, 7, 6)
recent_data: [[[0.25251926 0.10491998 0.11025489 0.14641375 0.11084766 0.        ]
  [0.25251926 0.10491998 0.11025489 0.14641375 0.10491998 0.5090809 ]
  [0.25429757 0.10491998 0.10966212 0.14641375 0.10491998 0.43368189]
  [0.25311203 0.33313574 0.10788382 0.14641375 0.10491998 0.33805724]
  [0.25133373 0.17308832 0.10788382 0.14641375 0.10491998 0.39529444]
  [0.24836989 0.3301719  0.10906935 0.14641375 0.10491998 0.50178866]
  [0.25074096 0.21754594 0.11025489 0.14641375 0.10491998 0.53783709]]]
test_predict1 [0.3957516]
내일의  visitors [[2876.3225]]
